## Imports

In [22]:
import pandas as pd
import folium

## Data Read-in

In [2]:
df = pd.read_csv('Eviction_Notices.csv')

## Make Map

In [28]:
df['File Date'] = pd.to_datetime(df['File Date'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_79166/4085003423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['File Date'] = pd.to_datetime(df['File Date'])


In [29]:
df = df[df['File Date'] >= '2023-01-01']

In [30]:
df['Location'].isna().value_counts()

Location
False    1380
Name: count, dtype: int64

In [31]:
df = df.dropna(subset='Location')

In [33]:
def parse_location(location_str):
    # Remove the parentheses and split the string into latitude and longitude
    lat, lon = map(float, location_str[1:-1].split(','))
    return pd.Series([lat, lon], index=['latitude', 'longitude'])

# Apply the function to the 'location' column and join the result with the original DataFrame
coordinates = df['Location'].apply(parse_location)
df = df.join(coordinates)

In [41]:
# Create a map object
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)
# Add markers to the map
for i, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(m)